In [1]:
import requests
import json
import pandas as pd
import sqlite3

In [2]:
year_list = []
for i in range(1996, 2021):
    year_list.append(str(i) + '-' + str(i+1)[-2:])
    
season_type_list = {'Regular+Season', 'Playoffs'}

print(season_type_list)

{'Playoffs', 'Regular+Season'}


In [3]:
def createResponseObj(year, season_type):
    player_info_url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=' + year + '&SeasonSegment=&SeasonType=' + season_type + '&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight='
    headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    }
    response = requests.get(url=player_info_url, headers=headers).json()
    return response
    
def createDFFromResponse(response):
    df = pd.json_normalize(response['resultSets'], record_path = 'rowSet') ##Grab the right selection from the json and puts it in rows
    df.columns = pd.json_normalize(response['resultSets'], record_path = 'headers')[0].tolist() ##Takes header columns and adds it to the dataframe
    return df

def createTableFromDF(df, tableName, dbName):
    connection = sqlite3.connect(dbName)
    statement = "drop table if exists " + tableName
    connection.execute(statement)
    df.to_sql(tableName, con=connection)
    connection.close()
    
def simplifySeasonTypeString(inputString):
    if inputString == 'Regular+Season':
        return 'regSeason'
    if inputString == 'Playoffs':
        return 'playoffs'
    

In [4]:
for year in year_list:
    for season_type in season_type_list:
        response = createResponseObj(year, season_type)
        df = createDFFromResponse(response)
        
        tableName = simplifySeasonTypeString(season_type) + year[:4] + '_' + year[-2:]
        createTableFromDF(df, tableName, r'nba.db')